<a href="https://colab.research.google.com/github/thaarres/Quantized_CNN/blob/master/kerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Run hyper parameter scan on SVHN**

First, install kerastuner and import the neccesary libraries


In [3]:
!pip install -U keras-tuner
import os
import IPython
import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds


Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


Set up TPUs

In [4]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.55.230.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.55.230.130:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Then define the model

In [5]:
def build_model(hp):
    """Builds a convolutional model."""
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    for i in range(hp.Int('conv_layers', 1, 3, default=3)):
        x = tf.keras.layers.Conv2D(
            filters=hp.Int('filters_' + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int('kernel_size_' + str(i), 3, 5),
            activation='relu',
            padding='same')(x)

        if hp.Choice('pooling' + str(i), ['max', 'avg']) == 'max':
            x = tf.keras.layers.MaxPooling2D()(x)
        else:
            x = tf.keras.layers.AveragePooling2D()(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

    if hp.Choice('global_pooling', ['max', 'avg']) == 'max':
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    model.compile(optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

Get the SVHN cropped dataset (32,32,3) streetnumber images

In [ ]:
def convert_dataset(item):
    image = item['image']
    label = item['label']
    image = tf.dtypes.cast(image, 'float32') / 255.
    return image, label

mnist_data = tfds.load('svhn_cropped')
ds_train, ds_test = ds_data['train'], ds_data['test']
ds_train = ds_train.map(convert_dataset).shuffle(1000).batch(100).repeat()
ds_test = ds_test.map(convert_dataset).batch(100)

Shuffling and writing examples to /root/tensorflow_datasets/svhn_cropped/3.0.0.incompleteO80PP4/svhn_cropped-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/svhn_cropped/3.0.0.incompleteO80PP4/svhn_cropped-test.tfrecord


Define the keras-tuner object to run the hyperparameter search

In [ ]:
tuner = kt.Hyperband(
        hypermodel=build_model,
        objective='val_accuracy',
        max_epochs=8,
        factor=2,
        hyperband_iterations=3,
        distribution_strategy=strategy,
        directory='results_hpscan,
        project_name='svhn_cropped_hpscan')

Before running the search, define a callback to clear  training outputs after every training step.

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Run scan

In [ ]:
 tuner.search(ds_train,
                 steps_per_epoch=600,
                 validation_data=ds_test,
                 validation_steps=100,
                 epochs=20,
                 callbacks=[ClearTrainingOutput(),tf.keras.callbacks.EarlyStopping('val_accuracy')])

Get the optimal hyperparameters

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of conv layers is {best_hps.get('conv_layers')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Then retrain using the best model!

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(ds_train, epochs = 10, validation_data = ds_test)